# Apache Iceberg Write Audit Publish (or Rollback) Pattern

## Create SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSession-Iceberg") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .getOrCreate()


spark

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d0fa5ff8-920f-4eff-9a78-5a8ecd3809b6;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.7.0/iceberg-spark-runtime-3.5_2.12-1.7.0.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0!iceberg-spark-runtime-3.5_2.12.jar (3688ms)
:: resolution report :: resolve 364ms :: artifacts dl 3692ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnl

In [2]:
spark.sql("DROP TABLE IF EXISTS iceberg.dunder_mifflin.dim_employee")

spark.sql("CREATE DATABASE IF NOT EXISTS dunder_mifflin")

Hive Session ID = fef67008-9708-4f51-8ac8-be4c8f698228


DataFrame[]

In [3]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS iceberg.dunder_mifflin.dim_employee (
    employee_id integer,
    first_name string,
    last_name string,
    department string,
    start_date date,
    end_date date
)
USING iceberg
LOCATION 's3a://lasagna-workspace-bronze/dunder_mifflin/dim_employee/'
"""
)

print(
    spark.sql(
        "SHOW CREATE TABLE iceberg.dunder_mifflin.dim_employee"
    ).collect()[0]["createtab_stmt"]
)

spark.sql(
"""
INSERT INTO iceberg.dunder_mifflin.dim_employee VALUES
    (1, 'Michael', 'Scott', 'Management', DATE '2005-03-24', NULL),
    (2, 'Jim', 'Halpert', 'Sales', DATE '2005-03-24', NULL),
    (3, 'Pam', 'Beesly', 'Reception', DATE '2005-03-24', NULL)
"""
)

spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee
"""
).show(truncate=False)

CREATE TABLE iceberg.dunder_mifflin.dim_employee (
  employee_id INT,
  first_name STRING,
  last_name STRING,
  department STRING,
  start_date DATE,
  end_date DATE)
USING iceberg
LOCATION 's3a://lasagna-workspace-bronze/dunder_mifflin/dim_employee'
TBLPROPERTIES (
  'current-snapshot-id' = 'none',
  'format' = 'iceberg/parquet',
  'format-version' = '2',
  'write.parquet.compression-codec' = 'zstd')



+-----------+----------+---------+----------+----------+--------+
|employee_id|first_name|last_name|department|start_date|end_date|
+-----------+----------+---------+----------+----------+--------+
|1          |Michael   |Scott    |Management|2005-03-24|NULL    |
|2          |Jim       |Halpert  |Sales     |2005-03-24|NULL    |
|3          |Pam       |Beesly   |Reception |2005-03-24|NULL    |
+-----------+----------+---------+----------+----------+--------+



25/03/25 22:05:24 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/03/25 22:05:24 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [ ]:
spark.sql(
"""
ALTER TABLE iceberg.dunder_mifflin.dim_employee
    SET TBLPROPERTIES ('write.wap.enabled'='true')
"""
)

spark.sql(
"""
SHOW TBLPROPERTIES iceberg.dunder_mifflin.dim_employee
"""
).show(truncate=False)

In [ ]:
spark.sql(
""" 
ALTER TABLE iceberg.dunder_mifflin.dim_employee
CREATE BRANCH IF NOT EXISTS `audit` RETAIN 7 DAYS
"""
)

spark.sql(
""" 
ALTER TABLE iceberg.dunder_mifflin.dim_employee
CREATE BRANCH IF NOT EXISTS `quarantine` RETAIN 7 DAYS
"""
)

spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.refs
"""
).show(truncate=False)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import date

new_data = [
    (1, "Michael", "Scott", "Management", date(2005, 3, 24), None), # Não muda nada
    (2, "Jim", "Halpert", "Management", date(2005, 3, 24), None), # Mudou de área
    (3, "Pam", "Beesly Halpert", "Reception", date(2005, 3, 24), None), # Mudou de nome
    (4, "Dwight", "Schrute", "Sales", date(2005, 4, 26), None) # Entrou na empresA
]

schema = StructType([
    StructField("employee_id", IntegerType(), nullable=False),
    StructField("first_name", StringType(), nullable=False),
    StructField("last_name", StringType(), nullable=False),
    StructField("department", StringType(), nullable=False),
    StructField("start_date", DateType(), nullable=False),
    StructField("end_date", DateType(), nullable=True)
])

new_df = spark.createDataFrame(new_data, schema)

new_df.show(truncate=False)

In [ ]:
spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).show(truncate=False)

last_snapshot = spark.sql(
"""
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).collect()[-1]['snapshot_id']

print(f"Last snapshot: {last_snapshot}")

In [ ]:
spark.conf.set("spark.wap.branch", "audit")

new_df.createOrReplaceTempView("new_dataframe")

spark.sql(
"""
MERGE INTO iceberg.dunder_mifflin.dim_employee AS OLD
USING new_dataframe AS NEW
ON OLD.employee_id = NEW.employee_id AND OLD.end_date IS NULL
WHEN MATCHED AND (
    NEW.first_name IS NOT NULL AND (
        OLD.first_name != NEW.first_name 
        OR OLD.last_name != NEW.last_name 
        OR OLD.department != NEW.department
        )
    ) THEN 
    UPDATE SET end_date = CURRENT_DATE()
"""
)

spark.sql(
"""
MERGE INTO iceberg.dunder_mifflin.dim_employee AS OLD
USING new_dataframe AS NEW
ON OLD.employee_id = NEW.employee_id AND OLD.end_date IS NULL
WHEN NOT MATCHED THEN
    INSERT (employee_id, first_name, last_name, department, start_date, end_date)
    VALUES (NEW.employee_id, NEW.first_name, NEW.last_name, NEW.department, CURRENT_DATE(), NULL)

"""
)

In [ ]:
# spark.read.option("BRANCH", "audit").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


# spark.read.option("BRANCH", "audit").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()

# spark.read.option("BRANCH", "quarantine").format("iceberg").load("iceberg.dunder_mifflin.dim_employee").show()

print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()

In [ ]:
is_good_quality = True

if is_good_quality:
    branch_to_merge = "main"
elif not is_good_quality:
    branch_to_merge = "quarantine"
    

spark.sql(f"""
CALL iceberg.system.fast_forward(
    'iceberg.dunder_mifflin.dim_employee',
    '{branch_to_merge}',
    'audit'
)
""").show()


print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()


print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()


In [ ]:
spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.snapshots
"""
).show(truncate=False)

In [ ]:
spark.sql(
f"""
CALL iceberg.system.rollback_to_snapshot(
    'iceberg.dunder_mifflin.dim_employee',
    {last_snapshot}
)
"""
).show(truncate=False)


is_good_quality = False

if is_good_quality:
    branch_to_merge = "main"
elif not is_good_quality:
    branch_to_merge = "quarantine"
    

spark.sql(f"""
CALL iceberg.system.fast_forward(
    'iceberg.dunder_mifflin.dim_employee',
    '{branch_to_merge}',
    'audit'
)
""").show()


print("branch :: [audit]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'audit'
"""
).show()


print("branch :: [main]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'main'
"""
).show()


print("branch :: [quarantine]")
spark.sql(
"""
SELECT *
FROM iceberg.dunder_mifflin.dim_employee
FOR VERSION AS OF 'quarantine'
"""
).show()

In [ ]:
spark.sql(
""" 
SELECT * 
FROM iceberg.dunder_mifflin.dim_employee.history
"""
).show(truncate=False)